### Merkle Tree


In [34]:
from hashlib import sha256

class MerkleTree:
    def __init__(self, leaves: list[int]):
        assert len(leaves) & (len(leaves) - 1) == 0, "Leaves must be a power of 2"
        self.leaves = leaves
        self.tree = []
        self.root = self._build_tree()

    def get_path(self, leaf_index: int) -> list[int]:
        path = []
        layer_index = 0
        path.append(self.tree[layer_index][leaf_index])
        while layer_index < len(self.tree) - 1:
            path.append(self.tree[layer_index][leaf_index + 1 if leaf_index % 2 else leaf_index - 1])
            layer_index += 1
            leaf_index = leaf_index // 2
        return path


    def _build_tree(self) -> list[int]:
        curr_layer = [sha256(str(leaf).encode()).hexdigest() for leaf in self.leaves]
        self.tree.append(curr_layer)
        while len(curr_layer) > 1:
            next_layer = self._build_layer(curr_layer)
            self.tree.append(next_layer)
            curr_layer = next_layer
        return curr_layer[0]
        

    def _build_layer(self, layer: list[int]) -> list[int]:
        new_layer = []
        for i in range(0, len(layer), 2):
            new_layer.append(sha256((layer[i]+ layer[i + 1]).encode()).hexdigest())
        return new_layer


In [33]:
from pprint import pprint

m = MerkleTree([1,2,3,4,5,6,7,8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a  in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35
['6b86', '2c62', 'ada1', 'e0e2']


### Hash Chain

In [3]:
import random

class HashChain:
    def __init__(self, seed: str):
        self.chain = [seed]
        self.random_generator = random.Random(seed)

    def get_next_random(self):
        nonce = self.random_generator.randbytes(4)
        self.chain.append(sha256(self.chain[-1].encode() + nonce).hexdigest())
        return int(self.chain[-1], 16)

    def get_all_hashes(self):
        return self.chain

### Channel

In [4]:
class Channel:
    def __init__(self, F):
        """Initialize the channel with an optional random seed."""
        self.messages = []
        self.hash_chain = None
        self.F = F

    def send(self, message):
        """Send (store) a message in the channel."""
        self.messages.append(message)

    def receive(self):
        """Receive (retrieve) the last message sent."""
        return self.messages[-1] if self.messages else None
    
    def get_all_messages(self):
        """Retrieve all messages exchanged."""
        return self.messages
    
    def receive_random_field_element(self):
        """Generate a random f"""
        if self.hash_chain is None:
            self.hash_chain = HashChain("".join(self.messages))
        random_field_element = self.F(self.hash_chain.get_next_random())
        self.messages.append(random_field_element)
        return random_field_element

### Fibonacci function

In [5]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list 

In [6]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [7]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8

g_group = [g ** i for i in range(N)]
gamma_group = [gamma ** i for i in range(1001)]

w = F.multiplicative_generator()

wg = [w * gi for gi in g_group]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f_x = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [8]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev

def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]

### First Commit

In [ ]:
f_x_on_wg = [f_x(x) for x in wg]

bit_reverse_order = bit_reverse_permutation(N)
f_x_on_wg_merkle_tree_order = [f_x_on_wg[i] for i in bit_reverse_order]
merkle_f_wg = MerkleTree(f_x_on_wg_merkle_tree_order)
original_commit =  merkle_f_wg.root

channel = Channel(F)
channel.send(original_commit)


[3095672467, 2847094869, 2027553343, 317299527, 709545447, 1233860353, 1193752494, 1746145883, 1323470169, 682508182, 2900620132, 249342471, 778403882, 1450348019, 843140049, 1746224955, 2349626404, 1521299731, 1848509917, 2340786591, 480514670, 2880392585, 2578676562, 3086343158, 1974819673, 2973470763, 967318275, 3161583568, 339413599, 91792319, 1267737621, 2992855102, 2322807770, 765295598, 1546724315, 1585500123, 62501662, 731533119, 676277648, 1934963510, 41400148, 778898076, 526505320, 3098491861, 1443918956, 1675504087, 1246101295, 3078917811, 2517175700, 1093634130, 1623383269, 2229363112, 1478334336, 2047798688, 2248644628, 1317174750, 1182503642, 2923286689, 252502243, 1337932427, 778005141, 354755223, 2779506823, 705373589, 3010047606, 2126375629, 1758711779, 1608755068, 1234549174, 2151456399, 290703995, 2922031591, 1327801831, 153191703, 2231489282, 1094571395, 48007432, 2493084140, 2143086817, 1797600855, 1475074016, 638201174, 2104613465, 795667042, 2005254152, 250204729

### Polynomial h(x)

In [10]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

print(d_x)

F_x = f_x(x) + f_x(gamma * x) - f_x((gamma**2) * x)

print(F_x)

h_x = F_x // d_x

print(h_x)

x^999 + 2493229384*x^998 + 1802362163*x^997 + 713611309*x^996 + 2103384791*x^995 + 2850688520*x^994 + 864586265*x^993 + 2933426998*x^992 + 2389331276*x^991 + 2986331512*x^990 + 109303957*x^989 + 2128257855*x^988 + 2630104275*x^987 + 2687807756*x^986 + 2435829647*x^985 + 1848739130*x^984 + 1663405657*x^983 + 1283490742*x^982 + 2240748881*x^981 + 304190183*x^980 + 3105034365*x^979 + 1344160383*x^978 + 157683337*x^977 + 1049261349*x^976 + 2312237523*x^975 + 1792962613*x^974 + 2136476384*x^973 + 350479409*x^972 + 755564107*x^971 + 2492357173*x^970 + 3931705*x^969 + 3126193360*x^968 + 2026670936*x^967 + 944155471*x^966 + 1563489142*x^965 + 2289741234*x^964 + 1044361898*x^963 + 197989197*x^962 + 2696752331*x^961 + 503152203*x^960 + 1090269794*x^959 + 840788646*x^958 + 1670137181*x^957 + 167992265*x^956 + 383212506*x^955 + 429437121*x^954 + 1057523094*x^953 + 2152704283*x^952 + 2318222108*x^951 + 942383880*x^950 + 776587224*x^949 + 1509445905*x^948 + 1325104074*x^947 + 1574599829*x^946 + 1428

### Boundary Constraints

In [11]:
Y = fibonacci_seq[-1]

t_0_x = (f_x(x) - 1) / (x - gamma ** 0)

t_1_x = (f_x(x) - Y) / (x - gamma ** 1000)

### FRI - Composition Polynomial

In [ ]:
channel = Channel(F)
channel.send(original_commit)
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

print(beta_0, beta_1, beta_2)

cp_0_x = beta_0 * h_x + beta_1 * t_0_x + beta_2 * t_1_x

# print(cp_0_x)

cp_0_on_wg = [cp_0_x(x) for x in wg]

cp_0_on_wg_reverse = [cp_0_on_wg[i] for i in bit_reverse_order]




cp_prev = R(cp_0_x)
print("channel before FRI", channel.get_all_messages())
while cp_prev.degree() > 0:
    random_x0 = channel.receive_random_field_element()
    cp_prev_on_wg = [cp_prev(i) for i in wg]

    merk = MerkleTree([cp_prev_on_wg[i] for i in bit_reverse_order])
    commit = merk.get_path(4)
    print(commit)

    channel.send(commit)
    
    cp_next = (cp_prev(x) + cp_prev(-x)) // 2 + random_x0 * (cp_prev(x) - cp_prev(-x)) // (2 * x)
    cp_next = sum(coef * x^(exp // 2) for exp, coef in R(cp_next).dict().items())
    cp_prev = cp_next
print('cp_prev:::::', cp_prev)



def verify_merkle_path(merkle_root: str, merkle_path: list[str]):
    root = merkle_path[0]
    for sibling in merkle_path[1:]:
        root = sha256((root + sibling).encode()).hexdigest()
    return root == merkle_root


def verify_basic():
    messages = channel.get_all_messages()
    print(messages)
    merkle_root_base = messages[0]
    random_number = messages[4]
    merkle_path = messages[5]
    print(merkle_path)
    print(verify_merkle_path(merkle_root_base, merkle_path))



verify_basic()

2166839664 93169854 1619602945
channel before FRI ['d524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56', 2166839664, 93169854, 1619602945]
cp_prev::::: 1963606400
['d524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56', 2166839664, 93169854, 1619602945, 2710878502, ['f50cc3d9e940db0c6de8a6fe16bae4aa6ae942180258aa74d9c1120ced2d77ca', 'bcb2d188050a42c682d873ae9e71e6aac3d893636ddfa1e616a592470fc31241', 'ac08acbffda579ad7eb2ac0923c6ffd0cb25eae1d68f03f69c0b08cb941577ad', 'd2286abf59cae4ade339433c52564f716a8150cb5da0a35330e8decc6f56511f', 'e5c443f393e2dd954041d276aabf1326381ce7ededdf7a7850572de7fa909869', '136eeb7621318692aa9595d6a33ca7c67a1e9bf6e7d2feefaf1b389cf0f9718a', '26681aeba9b2c008aac340a111b1e86bb75e7ceca512dc00692d23b753b51407', 'c7907b65b85578395c20f35e375e3822d98b5dbd1ace9681bb710976f3fe59e9', 'c850e897ddc839e8a07da68e5efdeed8fff6626c458d745691f0761e0c7b85b2', '4e5997f92b5a9950bf506be4adfdb18e9ad8a743bab52824cff8ab3e338a9bd3', '19eb085ca63907f84bde48c423